# SVM Classifier for Hard Negative Mining Project


In [28]:
# Run some setup code for this notebook.
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
#import lr_scheduler
import torch.optim.lr_scheduler

from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import operator
import numpy as np
import cv2
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

# This is to make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# So the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def applySIFT(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(gray, None)
    img = cv2.drawKeypoints(gray, kp, None)
    return kp

def extractSift(input_files):
    print ("extracting Sift features")
    all_features_dict = {}
    for i, fname in enumerate(input_files):
        features_fname = fname + '.sift'
        if exists(features_fname) == False:
            print ("calculating sift features for", fname)
            sift.process_image(fname, features_fname)
        print ("gathering sift features for", fname,
        locs, descriptors = sift.read_features_from_file(features_fname))
        print (descriptors.shape)
        all_features_dict[fname] = descriptors
    return all_features_dict

def dict2numpy(dict):
    nkeys = len(dict)
    array = zeros((nkeys * PRE_ALLOCATION_BUFFER, 128))
    pivot = 0
    for key in dict.keys():
        value = dict[key]
        nelements = value.shape[0]
        while pivot + nelements > array.shape[0]:
            padding = zeros_like(array)
            array = vstack((array, padding))
        array[pivot:pivot + nelements] = value
        pivot += nelements
    array = resize(array, (pivot, 128))
    return array


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## SIFT keypoints

In [29]:
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from feature_aggregation import BagOfWords

def sift(img):
    sift= cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(img, None)

def dsift(img, step=5):
    keypoints = [
        cv2.KeyPoint(x, y, step)
        for y in range(0, img.shape[0], step)
        for x in range(0, img.shape[1], step)
    ]
    features = sift().compute(img, keypoints)[1]
#     if (features.sum(axis=1).reshape(-1, 1)).any() == 0:
    features /= features.sum(axis=1).reshape(-1, 1)
    print(features)
    return features



In [27]:
# Generate dense SIFT features
data_dir_path = './facesDataAll'

# Iterate over data.
X_train = []
y_train = []
train_path = os.path.join(data_dir_path, 'train')
features = [];
for dir in os.listdir(train_path):
    if dir == 'attractive':
        label = 1
    elif dir == 'unattractive':
        label = 0
    else:
        continue
    subFolder_path = os.path.join(train_path, dir)
    for f in os.listdir(subFolder_path):
        if not f.startswith('.'):
            image = cv2.imread(os.path.join(subFolder_path, f))
#             (image.reshape(64, 64, 1)*255).astype(np.uint8)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


            keypoints = dsift(gray)
            features.append(keypoints)
#             X_train.append(np.asarray(keypointsSIFT))
            y_train.append(label)

X_dev = []
y_dev = []
dev_path = os.path.join(data_dir_path, 'dev')
for dir in os.listdir(dev_path):
    if dir == 'attractive':
        label = 1
    elif dir == 'unattractive':
        label = 0
    else:
        continue
    subFolder_path = os.path.join(dev_path, dir)
    for f in os.listdir(subFolder_path):
        if not f.startswith('.'):
            image = cv2.imread(os.path.join(subFolder_path, f))
#             (image.reshape(64, 64, 1)*255).astype(np.uint8)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


            keypoints = dsift(gray)
            features.append(keypoints)
#             print(os.path.join(subFolder_path, f))
#             print(type(image))
#             print(image.shape)
            keypointsSIFT = applySIFT(image)
            X_dev.append(np.asarray(keypointsSIFT))
            y_dev.append(label)

X_test = []
y_test = []
test_path = os.path.join(data_dir_path, 'test')
for dir in os.listdir(test_path):
    if dir == 'attractive':
        label = 1
    elif dir == 'unattractive':
        label = 0
    else:
        continue
    subFolder_path = os.path.join(test_path, dir)
    for f in os.listdir(subFolder_path):
        if not f.startswith('.'):
            image = cv2.imread(os.path.join(subFolder_path, f))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            keypoints = dsift(gray)
            features.append(keypoints)
            
            keypointsSIFT = applySIFT(image)
            X_test.append(np.asarray(keypointsSIFT))
            y_test.append(label)

In [31]:
# Aggregate those features with bag of words using online training
print(features)
bow = BagOfWords(100)
for i in range(2):
    for j in range(0, len(features), 10):
        bow.partial_fit(features[j:j+10])
faces_bow = bow.transform(features)

# Split in training and test set
train = np.arange(len(features))
np.random.shuffle(train)
test = train[200:]
train = train[:200]

# Train and evaluate
svm = Pipeline([("chi2", AdditiveChi2Sampler()), ("svm", LinearSVC(C=10))])
svm.fit(faces_bow[train], faces.target[train])
print(classification_report(faces.target[test], svm.predict(faces_bow[test])))

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

AttributeError: 'NoneType' object has no attribute 'shape'

In [6]:
## Fine SIFT Keypoints in all images and use as our data

data_dir_path = './facesDataAll'

# Iterate over data.
X_train = []
y_train = []
train_path = os.path.join(data_dir_path, 'train')
for dir in os.listdir(train_path):
    if dir == 'attractive':
        label = 1
    elif dir == 'unattractive':
        label = 0
    else:
        continue
    subFolder_path = os.path.join(train_path, dir)
    for f in os.listdir(subFolder_path):
        if not f.startswith('.'):
            image = cv2.imread(os.path.join(subFolder_path, f))
            keypointsSIFT = applySIFT(image)
            X_train.append(np.asarray(keypointsSIFT))
            y_train.append(label)

X_dev = []
y_dev = []
dev_path = os.path.join(data_dir_path, 'dev')
for dir in os.listdir(dev_path):
    if dir == 'attractive':
        label = 1
    elif dir == 'unattractive':
        label = 0
    else:
        continue
    subFolder_path = os.path.join(dev_path, dir)
    for f in os.listdir(subFolder_path):
        if not f.startswith('.'):
            image = cv2.imread(os.path.join(subFolder_path, f))
#             print(os.path.join(subFolder_path, f))
#             print(type(image))
#             print(image.shape)
            keypointsSIFT = applySIFT(image)
            X_dev.append(np.asarray(keypointsSIFT))
            y_dev.append(label)

X_test = []
y_test = []
test_path = os.path.join(data_dir_path, 'test')
for dir in os.listdir(test_path):
    if dir == 'attractive':
        label = 1
    elif dir == 'unattractive':
        label = 0
    else:
        continue
    subFolder_path = os.path.join(test_path, dir)
    for f in os.listdir(subFolder_path):
        if not f.startswith('.'):
            image = cv2.imread(os.path.join(subFolder_path, f))
            keypointsSIFT = applySIFT(image)
            X_test.append(np.asarray(keypointsSIFT))
            y_test.append(label)



In [7]:
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = np.array(X_train)
y_train = np.array(y_train)

X_dev = np.array(X_dev)
y_dev = np.array(y_dev)

In [8]:


# Flatten and convert data into rows as part of out preprocess
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

#reshape y's
y_train = np.reshape(y_train, (X_train.shape[0], 1))
y_test = np.reshape(y_test, (X_test.shape[0], 1))
y_dev = np.reshape(y_dev, (X_dev.shape[0], 1))

#Sanity Check
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)

print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)


print('Dev data shape: ', X_dev.shape)
print('Dev labels shape: ', y_dev.shape)

Train data shape:  (948, 1)
Train labels shape:  (948, 1)
Test data shape:  (183, 1)
Test labels shape:  (183, 1)
Dev data shape:  (180, 1)
Dev labels shape:  (180, 1)


In [9]:
# Normalizing
mean_image = np.mean(X_train, axis=0)

X_train -= mean_image
# X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

ValueError: operands could not be broadcast together with shapes (238,) (129,) 

In [ ]:
# Add bias
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
# X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

# print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)
print(X_train.shape, X_test.shape, X_dev.shape)

### SVM

Compute SVM to minimize the loss.

In [ ]:
# from utils.classifiers import LinearSVM
# svm = LinearSVM()
# tic = time.time()
# loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,
#                       num_iters=1500, verbose=True)
# toc = time.time()
# print('That took %fs' % (toc - tic))

from sklearn.svm import SVC
clf = SVC()
tic = time.time()
clf.fit(X_train, y_train.ravel()) 
toc = time.time()
print('That took %fs' % (toc - tic))


test_acc = clf.score(X_test, y_test.ravel())
train_acc = clf.score(X_train, y_train.ravel())
print('Train acc: ', train_acc)
print('Test acc: ', test_acc)